In [1]:
from honeybee_plus.room import Room
from honeybee_plus.radiance.material.glass import Glass
# from honeybee_plus.radiance.sky.certainIlluminance import CertainIlluminanceLevel
# from honeybee_plus.radiance.recipe.pointintime.gridbased import GridBased
from honeybee_plus.radiance.recipe.solaraccess.gridbased import SolarAccessGridBased
from honeybee_plus.radiance.analysisgrid import AnalysisGrid

Path to radiance is set to: /usr/local/radiance
Path to perl is set to: /usr/bin


In [2]:
# create a test room
room = Room(origin=(0, 0, 3.2), width=4.2, depth=6, height=3.2,
            rotation_angle=45)

# add fenestration
# add a window to the back wall
room.add_fenestration_surface(
    wall_name='back', width=2, height=2, sill_height=0.7)

# add another window with custom material. This time to the right wall
glass_60 = Glass.by_single_trans_value('tvis_0.6', 0.6)
room.add_fenestration_surface('right', 4, 1.5, 1.2, radiance_material=glass_60)

In [3]:
# # generate the sky
# sky = CertainIlluminanceLevel(illuminance_value=2000,sky_type=1)
# print(CertainIlluminanceLevel.__doc__)

In [4]:
# # generate grid of test points
# analysis_grid = room.generate_test_points(grid_size=0.5, height=0.75)
test_pts = [(0, 0, 0), (1, 1, 0), (3, 2, 0)]
test_vec = [(0, 0, 1), (0, 0, 1), (0, 0, 1)]
analysis_grid = AnalysisGrid.from_points_and_vectors(test_pts, test_vec, 'test_grid')

# define sun vectors
sun_vectors = [
            (-0.810513, 0.579652, -0.084093), (-0.67166, 0.702357, -0.235729),
            (-0.487065, 0.798284, -0.354275), (-0.269301, 0.8609, -0.431657),
            (-0.033196, 0.885943, -0.462605), (0.20517, 0.871705, -0.445013),
            (0.429563, 0.819156, -0.380077), (0.624703, 0.731875, -0.272221),
            (0.777301, 0.615806, -0.128788)]

# define sun hours : A list of hours of the year for each sun vector
# there are 87600 hours in a year, so the following integers refer to specific hours throughout the year
hoys = [1908, 2000, 2530, 3254, 3658, 4000, 4116, 6324, 8508]

# # put the recipe together
# rp = GridBased(sky=sky, analysis_grids=(analysis_grid,), simulation_type=0, hb_objects=(room,))
rp = SolarAccessGridBased(sun_vectors, hoys, [analysis_grid], hb_objects=(room,))

Found 6 opaque surfaces.
Found 4 fenestration surfaces.
Found 0 window-groups.


In [5]:
# write simulation to folder
batch_file = rp.write(
    target_folder='.', project_name='room_sun')

Writing recipe contents to: ./room_sun/solaraccess


In [6]:
# run the simulation
for com in rp._commands:
    print(com)
rp.run(batch_file, debug=False)

# https://www.radiance-online.org/learning/documentation/manual-pages/pdfs/rmtxop.pdf
# Eleonora: rmtxop multiplies the daylight coefficient matrices with the sky vectors. This is necessary for getting illuminance from daylight coefficients computed w.r.t. rgb components. 
# Sun Access of points, and how to interpret them https://floyd.lbl.gov/radiance/refer/usman2.pdf

#!/usr/bin/env bash



cd ./room_sun/solaraccess

/usr/local/radiance/bin/oconv -f scene/opaque/room_sun..opq.mat scene/opaque/room_sun..opq.rad scene/glazing/room_sun..glz.mat scene/glazing/room_sun..glz.rad sky/analemma.rad > room_sun.oct
/usr/local/radiance/bin/rcontrib -aa 0.0 -ab 0 -ad 512 -ar 16 -as 128 -dc 1.0 -dj 0.0 -dp 64 -ds 0.5 -dr 0 -dt 0.0 -I -lr 4 -lw 0.05 -M ./sky/analemma.mod -ss 0.0 -st 0.85 -y 3 room_sun.oct < room_sun.pts > result/room_sun.dc
/usr/local/radiance/bin/rmtxop -c 47.4 119.9 11.6 -fa result/room_sun.dc > result/room_sun.ill


True

In [7]:
# results - in this case it will be an analysis grid
result = rp.results()[0]
print(rp.results()[0])
# print the values for each point
for value in result.combined_value_by_id():
    print(value)

Unloading the current values from the analysis grids.
Unloading the current values from the analysis grids.
0.000 0.000 0.000 0.000 0.000 1.000
1.000 1.000 0.000 0.000 0.000 1.000
3.000 2.000 0.000 0.000 0.000 1.000
(1, None)
(1, None)
(1, None)
